In [9]:
import os
import besos
from besos import sampling
from besos import eppy_funcs as ef
from besos import eplus_funcs as ep
from besos.problem import EPProblem
from besos.evaluator import EvaluatorEP
from besos.parameters import wwr, RangeParameter, FieldSelector, FilterSelector, GenericSelector, Parameter, expand_plist, DependentParameter, CategoryParameter
from besos.objectives import MeterReader,time_series_values,EPResults
from besos.optimizer import NSGAII, df_solution_to_solutions
import pandas as pd
import matplotlib.pyplot as plt
from seaborn import heatmap
from seaborn import pairplot
import numpy as np

# Open the IDF file
idf_1 = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/1_layer_window.idf"
idf_2 = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/2_layer_window.idf"
idf_3 = "/home/ict4bd/Desktop/ict4building/DesignBuilderModel/3_layer_window.idf"
idfs = [idf_1,idf_2,idf_3]
weather_file = '/home/ict4bd/Desktop/ict4building/Requirements/CHE_Geneva.067000_IWEC/CHE_Geneva.067000_IWEC.epw'
## Set up evaluator
# Load Building
building = ef.get_building(idf_2)

# Parameter Space
parameters = []

orientation = FieldSelector(class_name='Building', field_name='North Axis')
orientation_descriptor = CategoryParameter(options=[0, 90, 180, 270])

insulation = FieldSelector(class_name='Material', object_name='XPS Extruded Polystyrene  - CO2 Blowing_.O792', 
                           field_name='Thickness')
insulation_descriptor = RangeParameter(min_val = 0.01, max_val=0.5) 

shading = FieldSelector(class_name='WindowShadingControl',field_name='Setpoint')
shading_descriptor = RangeParameter(min_val = 0, max_val=30) 

ventilation = FieldSelector(class_name='ZoneVentilation:DesignFlowRate',field_name='Design Flow Rate')
ventilation_descriptor = CategoryParameter(options=[0, 0.12])




parameters.append(Parameter(selector=orientation, 
                              value_descriptor=orientation_descriptor,
                              name='Orientation'))

parameters.append(Parameter(selector=ventilation,
                            value_descriptor=ventilation_descriptor,
                            name='Ventilation'))

parameters.append(Parameter(selector=shading,
                            value_descriptor=shading_descriptor,
                            name='Shading'))

parameters.append(Parameter(selector=insulation,
                            value_descriptor=insulation_descriptor,
                            name='Insulation Thickness'))

parameters.append(wwr(RangeParameter(0.1,0.9))) 

# Objectives of our problem which is to minimize the TOTAL COMSUMPTION
objectives = ['Electricity:Facility','DistrictCooling:Facility', 'DistrictHeating:Facility']

# problem = parameters + objectives
besos_problem = EPProblem(parameters, objectives)

# EPProblem vs EvaluatorEP = problem + building + weather
evaluator = EvaluatorEP(besos_problem, building, out_dir='outputdir', err_dir='outputdir', epw_file=weather_file) 

samples_temp = []
orientation_values = [0,90,180,270] 
wwr_values = [.15, .5, .9] 
ventilation_values = [0, 0.12] 
tickness_values = np.linspace(0.01,0.5,5)
shading = np.linspace(5,25,6)
for ori in orientation_values:
    for wwr_ in wwr_values:
        for vent in ventilation_values:
            for tickness in tickness_values:
                for shade in shading:
                    samples_temp.append({'Orientation': ori,
                                         'Ventilation': vent,
                                         'Shading': shade,
                                         'Insulation Thickness': tickness,
                                         'Window to Wall Ratio': wwr_,})
samples = pd.DataFrame.from_dict(samples_temp)
samples

,Orientation,Ventilation,Shading,Insulation Thickness,Window to Wall Ratio
0,0,0.00,20.0,0.01,0.15
1,0,0.00,21.0,0.01,0.15
2,0,0.00,22.0,0.01,0.15
3,0,0.00,23.0,0.01,0.15
4,0,0.00,24.0,0.01,0.15
...,...,...,...,...,...
715,270,0.12,21.0,0.50,0.90
716,270,0.12,22.0,0.50,0.90
717,270,0.12,23.0,0.50,0.90
718,270,0.12,24.0,0.50,0.90


In [10]:
# return dataframe with results described in objectives
outputs = evaluator.df_apply(samples, keep_input=True)

Program Version,EnergyPlus, Version 9.0.1-bb7ca4f0da, YMD=2020.12.24 21:36,

   ** Warning ** Timestep: Requested number (2) is less than the suggested minimum of 4.

   **   ~~~   ** Please see entry for Timestep in Input/Output Reference for discussion of considerations.

   ************* Beginning Zone Sizing Calculations

   ** Warning ** GetSurfaceData: Construction PROJECT PARTITION of interzone surface BLOCK1:ZONE3_PARTITION_2_0_0 does not have the same materials in the reverse order as the construction PROJECT PARTITION_REV of adjacent surface BLOCK1:ZONE1_PARTITION_4_0_10004

   **   ~~~   ** ...but Nominal U values are similar, diff=[0.0000] ... simulation proceeds.

   **   ~~~   ** ...if the two zones are expected to have significantly different temperatures, the proper "reverse" construction should be created.

   **   ~~~   ** ...this problem for this pair will not be reported again.

   ** Warning ** GetSurfaceData: Entered Zone Floor Areas differ from calculated Zone Fl

/home/ict4bd/py3/lib/python3.8/site-packages/besos/evaluator.py:179: UserWarning: for inputs: ['Orientation', 'Ventilation', 'Shading', 'Insulation Thickness', 'Window to Wall Ratio']problematic values were: (0.0, 0.0, 22.0, 0.5, 0.9)
  warn(msg)


CalledProcessError: Command '['/usr/local/EnergyPlus-9-0-1/energyplus', '--idd', '/usr/local/EnergyPlus-9-0-1/Energy+.idd', '--weather', '/home/ict4bd/Desktop/ict4building/Requirements/CHE_Geneva.067000_IWEC/CHE_Geneva.067000_IWEC.epw', '--output-directory', '/home/ict4bd/Desktop/ict4building/Optimization/outputdir/BESOS_Output', '/home/ict4bd/.besos_527p9b37/in.idf']' returned non-zero exit status 1.

In [6]:
# Q is kWh / m2
total_area= 0
for Zones in building.idfobjects["Zone"]:
    total_area += float(Zones.checkrange("Floor_Area")) 
outputs['total (Q)'] = outputs[objectives].sum(axis=1) /3.6e6 /total_area 
outputs[outputs['total (Q)']==outputs['total (Q)'].min()]

,Orientation,Ventilation,Shading,Insulation Thickness,Window to Wall Ratio,Electricity:Facility,DistrictCooling:Facility,DistrictHeating:Facility,total (Q)
1,90,0.12,20,0.5,0.5,5.772262e+09,0.0,1.335069e+10,113.3483
